# Chap2. 자연어와 단어의 분산 표현
## 2.4 통계 기반 기법 개선하기

앞 절에서 생성한 동시 발생 행렬(co-occurence matrix) 개선 및 실용적인 말뭉치를 사용한 '진짜'단어의 분산 표현 구현하기

### 2.4.1 상호정보량

앞서 살펴본 co-occurence matrix에서 행렬의 원소는 두 단어가 동시에 발생한 횟수를 나타낸다. 그러나 이 '발생 횟수'는 사실 그리 좋은 특징이 아니다.

문서에서 'the car'라는 문구가 자주보인다.그렇다면 'the'와 'car'의 동시 발생 횟수 또한 높을 것이다.<br>
한편 'car'와 'drive'는 확실히 관련이 깊다. 하지만 단순 등장 횟수(동시 발생 횟수 등)만 보면 'car'는 'drive'보다는 'the'와의 관련성이 훨씬 강하다고 나올 것이다.

이 문제를 해결하기 위해 점별 상호정보량(Pointwise Mutual Information(PMI))라는 척도를 사용한다.<br>
(확률에 관해서는 3.5.1 CBOW 모델과 확률 절에서 자세히 설명한다)

In [2]:
# library import

import numpy as np

In [9]:
## Do-it common/utils ppmi()

def ppmi(C, verbose=False, eps=1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0
    
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i,j] * N / (S[j]*S[i]) + eps)  # ??
                                                           # S[j] 전체 corpus에서 j가 발생한 횟수
                                                           # C(j)
                                                           # S[i]도 마찬가지이다.
            M[i, j] = max(0, pmi)
            
            if verbose:
                cnt += 1
                if cnt % (total//100) == 0:
                    print('%.1f%%완료' % (100*cnt/total))


---
Break! common/utils.py 구현하고 오기 

---

In [8]:
a = np.array([[0, 1, 2, 3],
             [1, 2, 3, 4]])

np.sum(a, axis=1)

array([ 6, 10])

### 2.4.2 차원 감소

### 2.4.3 SVD에 의한 차원 감소

### 2.4.4 PTB 데이터셋

### 2.4.5 PTB 데이터셋 평가